In [ ]:
# 需要修改的代码包括
# 1 读写 裁剪TIFF （完成）
# 2 随机移动 （完成）
# 3 制作pix2pix样本（水平连接两个基本类型样本）（完成）

# Read, Clip, and Write GeoTIFF
## file path = E:\Backup\CodeBackup\1AboutCNN\1preprocess\General\1TiffReadClipWrite.py

In [1]:
import numpy as np
from osgeo import gdal
import os
import imageio
import glob

# from General.TiffReadClipWrite import clipgeotiff

In [9]:
# in 1TiffReadClipWrite.py
'''
readtiff2array: read tif file and output an numpy array
calculateTransform: calculating 'geotrans' affine transformation 
clipgeotiff_topatch: clip the complete data to small patches. (nonrandom)
'''
def readtiff2array(oriPath):
    in_ds = gdal.Open(oriPath)
    print("open tif file succeed")

    # 读取原图中的每个波段
    row = in_ds.RasterYSize  # 行
    col = in_ds.RasterXSize  # 列
    band = in_ds.RasterCount  # 波段
    geoTrans = in_ds.GetGeoTransform()
    geoPro = in_ds.GetProjection()

    # specific datatype
#     newDataType = gdal.GDT_UInt16;
#     data = np.zeros([row, col, band], newDataType)  # 建立数组保存读取的tiff

    
    # according to the input datatype
    datatype_index = in_ds.GetRasterBand(1).DataType
    datatype = gdal.GetDataTypeName(datatype_index)
#     print(datatype)
    if 'GDT_Byte' in datatype:
        newDataType = 'int8'
    elif 'GDT_UInt16' in datatype:
        newDataType = 'int16'
    else:
        newDataType = 'float32'
    data = np.zeros([row, col, band], newDataType)  # 建立数组保存读取的tiff

    for i in range(band):
        dt = in_ds.GetRasterBand(i + 1)
        # 从每个波段中裁剪需要的矩形框内的数据
        data[:, :, i] = dt.ReadAsArray(0, 0, col, row)

    del in_ds
    
    return data, [band, datatype, geoTrans, geoPro]
    # data shape = HWC
'''
16bit to 8bit
for slope data, nodata value = 91, therefore we need to find a maximum except 91 and view it as the max_16bit
'''
def transfer_16bit_to_8bit(data, isSlope=False, isDEM=False, nodatavalue=91):
    # transform to 8bit

    
    # 以下两部分主要是为了处理无效值，要将无效值区域调至max
    
    ## slope 数据中无效值被设置为了91
    if isSlope is True:
        
        min_16bit = 50000
        max_16bit = 0
    
        # 先对有效值区域进行处理，找到有效值的最大最小值
        ## 找到有效值的所有索引
        indexLS = np.argwhere(data<nodatavalue)
        ## indexLS为tuple，shape=[n,3],n代表有多少行 
        ## 具体在图像中的坐标为 indexLS.shape[0][0], indexLS.shape[0][1]
        rows = indexLS.shape[0]
        i = 0
        for i in range(rows):
            ## 找到其临时值
            tempValue = data[indexLS[i][0],indexLS[i][1],0]
            ## 对临时值进行判断
            if tempValue != 91:
                if tempValue > max_16bit:
                    max_16bit = tempValue
                elif tempValue < min_16bit:
                    min_16bit = tempValue      

        # 进而对无效值（通常为91）进行处理，思路为将其改为最大值+最大值的10%
        # 循环过程与上一步类似
        indexGE = np.argwhere(data>=nodatavalue)    
        rows = indexGE.shape[0]
        i = 0
        for i in range(rows):
            data[indexGE[i][0],indexGE[i][1],0] = max_16bit #+ max_16bit*0.1
        print(max_16bit, min_16bit)
        
    ## dem数据中无效值被设置为了0
    elif isDEM is True:
        nodatavalue = 0
        min_16bit = 50000
        max_16bit = 0
        
        # 先对有效值区域进行处理，找到有效值的最大最小值
        ## 找到有效值的所有索引
        indexGS = np.argwhere(data>nodatavalue)
        ## indexGS为tuple，shape=[n,3],n代表有多少行 
        ## 具体在图像中的坐标为 indexLS.shape[0][0], indexLS.shape[0][1]
        rows = indexGS.shape[0]
        i = 0
        for i in range(rows):
            ## 找到其临时值
            tempValue = data[indexGS[i][0],indexGS[i][1],0]
            ## 对临时值进行判断
            if tempValue != 0:
                if tempValue > max_16bit:
                    max_16bit = tempValue
                elif tempValue < min_16bit:
                    min_16bit = tempValue      

        # 进而对无效值（通常为0）进行处理，思路为将其改为最大值+最大值的10%
        # 循环过程与上一步类似
        indexLE = np.argwhere(data<=nodatavalue)    
        rows = indexLE.shape[0]
        i = 0
        for i in range(rows):
            data[indexLE[i][0],indexLE[i][1],0] = max_16bit #+ max_16bit*0.1
        print(max_16bit, min_16bit)
    else:   
        min_16bit = np.min(data)
        max_16bit = np.max(data)
    
    data_8bit = np.array(np.rint(255 * ((data - min_16bit) / (max_16bit - min_16bit))), dtype=np.uint8)
    return data_8bit
'''
仿射变换
'''
def calculateTransform(ori_transform, offsetX, offsetY):
    # 读取原图仿射变换参数值
    top_left_x = ori_transform[0]  # 左上角x坐标
    w_e_pixel_resolution = ori_transform[1]  # 东西方向像素分辨率
    top_left_y = ori_transform[3]  # 左上角y坐标
    n_s_pixel_resolution = ori_transform[5]  # 南北方向像素分辨率

    # 根据反射变换参数计算新图的原点坐标
    top_left_x = top_left_x + offsetX * w_e_pixel_resolution
    top_left_y = top_left_y + offsetY * n_s_pixel_resolution

    # 将计算后的值组装为一个元组，以方便设置
    dst_transform = (top_left_x, ori_transform[1], ori_transform[2], top_left_y, ori_transform[4], ori_transform[5])

    return dst_transform
'''
裁剪主函数 调用topatch
typeName & index: 输入数据编号
'''
def clipgeotiff(inputPath, savePath, patchSize, patchIntersection, startCol, startRow, typeName, index, To8bit=False, isSlope=False,isDEM=False):
    os.makedirs(savePath, exist_ok=True)
    # 遍历文件夹
    from_names = glob.glob(os.path.join(inputPath, "*.tif"))

    # 左上角坐标移动步长
    stride = patchSize - int((patchSize - patchIntersection) / 2)

    for i in range(len(from_names)):
        print(from_names[i])
        filePath = os.path.join(inputPath, os.path.basename(from_names[i]))
        # oriPara=[band, datatype, geoTrans, geoPro]
        inputData, oriPara = readtiff2array(filePath)

        # 16bit to 8bit
        if To8bit is True:
            inputData = transfer_16bit_to_8bit(inputData, isSlope,isDEM)

        # 裁剪 起始坐标
        # 原始代码的[offsetY, offsetX] = [offsetRow, offsetCol]
        offsetCor = [startRow, startCol]
        while offsetCor[1] < inputData.shape[1] - patchSize:
            while offsetCor[0] < inputData.shape[0] - patchSize:
                filename = typeName + str(index) + '_' + str(offsetCor[1]) + '_' + str(offsetCor[0]) + '.tif'
                saveName = os.path.join(savePath, filename)
                cliptopatch(inputData, oriPara[2], oriPara[3], saveName, offsetCor, patchSize)
                offsetCor[0] = offsetCor[0] + stride
            offsetCor[1] = offsetCor[1] + stride
            offsetCor[0] = 0
        index = index + 1

'''
裁剪至小块 并保存
inputData: array
offsetCor: patch的左上角位置 原始代码的[offsetY, offsetX] = [offsetRow, offsetCol]
saveName
patchSize
'''
def cliptopatch(inputData, ori_geoTrans, ori_geoPro, saveName, offsetCor, patchSize):
    # 读取输入数据信息
    ori_Datatype = inputData.dtype
    # inputData
    if len(inputData.shape) == 3:
        in_bands = inputData.shape[2]
    else:
        in_bands = 1

    # 读取要裁剪的原图
    out_band = np.zeros([patchSize, patchSize, in_bands], ori_Datatype)
    # print(out_band.shape)
    for i in range(in_bands):
        out_band[:, :, i] = inputData[offsetCor[0]:offsetCor[0] + patchSize, offsetCor[1]:offsetCor[1] + patchSize, i]

    # 获取原图的原点坐标信息
    ori_transform = ori_geoTrans
    # 计算仿射变化参数
    dst_transform = calculateTransform(ori_transform, offsetCor[1], offsetCor[0])
    
    # 设置DataType
    if 'int8' in out_band.dtype.name:
        newDataType = gdal.GDT_Byte
    elif 'int16' in out_band.dtype.name:
        newDataType = gdal.GDT_UInt16
    else:
        newDataType = gdal.GDT_Float32

    # 创建gtiff 并 写入
    driver = gdal.GetDriverByName("GTiff")
    dataset = driver.Create(saveName, patchSize, patchSize, in_bands, newDataType)
    if (dataset != None):
        dataset.SetGeoTransform(dst_transform)  # 写入仿射变换参数
        dataset.SetProjection(ori_geoPro)  # 写入投影
    for i in range(in_bands):
        dataset.GetRasterBand(i + 1).WriteArray(out_band[:, :, i])
    del dataset

    print("End!")

In [10]:
inputPath = "E:/Dataset/202101_withDinghu/temp/subAreas/V3/test/dem"
# "E:/Dataset/202101_withDinghu/temp/subAreas/V3/train/label"
#"E:/Dataset/202101_withDinghu/temp/subAreas/V3/train/dem"
# "E:/Dataset/202101_withDinghu/temp/wholeArea/Pujia/slope"
savePath = "E:/Dataset/202101_withDinghu/temp/Dataset/202202datasetV1_256/wangmaogou/allpatch/test/dem16bit_crop"
# "E:/Dataset/202101_withDinghu/temp/Dataset/202202datasetV1_256/wangmaogou/allpatch/train/mask_crop"
# "E:/Dataset/202101_withDinghu/temp/Dataset/202202datasetV1_256/wangmaogou/allpatch/train/dem8bit_crop" # wangjiamao
# "E:/Dataset/202101_withDinghu/temp/Dataset/202202datasetV1_256/pujia/allpatch/dem8bit_crop" # pujia
patchSize = 256
patchIntersection = 200
startCol = 0
startRow = 0
typeName = 'W'
index = 0
clipgeotiff(inputPath, savePath, patchSize, patchIntersection, startCol, startRow, typeName, index, To8bit=False, isSlope=False,isDEM=False)

E:/Dataset/202101_withDinghu/temp/subAreas/V3/test/dem\Wang_dem_fortrain.tif
open tif file succeed
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
End!
E

# Concatenate two samples
## producing dataset for pix2pix

In [36]:
# 暂时不成功 运行下一段可行
from General.ConcatenateSamples import concate_samples
# from General.TiffReadClipWrite import write_to_disk

In [11]:
import numpy as np
from osgeo import gdal
import os
import imageio
import glob

In [28]:
# in ConcatenateSamples.py
def concate_samples(sample1, sample2, savepath, filename):
    sample1 = sample1.reshape((1, sample1.shape[0], sample1.shape[1], sample1.shape[2]))
    sample2 = sample2.reshape((1, sample2.shape[0], sample2.shape[1], sample2.shape[2]))

    sample1band = sample1.shape[3]
    sample2band = sample2.shape[3]
    print(sample1band, sample2band)
    print(sample1.dtype, sample2.dtype)

    # if two samples bands == 1, then firstly concatenating them and secondly repeating it
    if sample1band == 1 and sample2band == 1:
        temp = np.concatenate((sample1, sample2), axis=2)
        temp = temp.repeat(3, axis=3)
    # if two samples bands == 3, then concatenating them
    elif sample1band == 3 and sample2band == 3:
        temp = np.concatenate((sample1, sample2), axis=2)
    elif sample1band == 3 and sample2band == 1:
        sample2 = sample2.repeat(3, axis=3)
        temp = np.concatenate((sample1, sample2), axis=2)
    elif sample1band == 1 and sample2band == 3:
        sample1 = sample1.repeat(3, axis=3)
        temp = np.concatenate((sample1, sample2), axis=2)
    else:
        print("The situation is not defined.")

    out_data = np.squeeze(temp, axis=0)
#     out_data = out_data.astype(np.uint16)  # data type
    print(out_data.shape)
    write_to_disk(savepath, out_data, filename)
# in TiffReadClipWrite.py
def write_to_disk(savePath, img_data, from_names, over_write=False):

    os.makedirs(savePath, exist_ok=True)

    full_name = from_names.replace(".tif", ".png")
    print(img_data.dtype)
    save_filePath = os.path.join(savePath, full_name)
    
#    im = Image.fromarray(np.uint16(img_data))
#    im.save(save_filePath) #--> 16bit
    
    print(save_filePath)
    imageio.imwrite(save_filePath,img_data, 'PNG-FI') #(save_filePath, img_data)  #imageio.imwrite
    print("Done!")

In [30]:
sample1Path = "E:/Dataset/202101_withDinghu/temp/Dataset/202202datasetV1_256/wangmaogou/selectedpatch/test/image_crop"
# "E:/Dataset/202101_withDinghu/temp/Dataset/202201datasetV3/size224/s2/valid/image_crop"
sample2Path = "E:/Dataset/202101_withDinghu/temp/Dataset/202202datasetV1_256/wangmaogou/selectedpatch/test/dem16bit_crop"
# "E:/Dataset/202101_withDinghu/temp/Dataset/202201datasetV3/size224/s2/valid/slope_crop"
savePath = "E:/Dataset/202101_withDinghu/temp/Dataset/202202datasetV1_256/wangmaogou/selectedpatch/test/basicID16_crop"
# "E:/Dataset/202101_withDinghu/temp/Dataset/202201datasetV3/size224/s2/valid/GANpatches"
os.makedirs(savePath, exist_ok=True)

# 遍历文件夹
from_names = glob.glob(os.path.join(sample1Path, "*.tif"))
for i in range(len(from_names)):
    basename = os.path.basename(from_names[i])
    
    file1Path = os.path.join(sample1Path, basename)
    # oriPara=[band, datatype, geoTrans, geoPro]
    input1Data, oriPara = readtiff2array(file1Path)
    
    file2Path = os.path.join(sample2Path, basename)
    # oriPara=[band, datatype, geoTrans, geoPro]
    input2Data, oriPara = readtiff2array(file2Path)
    
    # 保存16bit DEM 需要打开下两行
#     input1Data = input1Data.astype(np.uint16)
#     input2Data = input2Data.astype(np.uint16)
    
    concate_samples(input1Data, input2Data, savePath, basename)

open tif file succeed
open tif file succeed
3 1
uint16 uint16
(256, 512, 3)
uint16
E:/Dataset/202101_withDinghu/temp/Dataset/202202datasetV1_256/wangmaogou/selectedpatch/test/basicID16_crop\W0_1596_2280.png
Done!
open tif file succeed
open tif file succeed
3 1
uint16 uint16
(256, 512, 3)
uint16
E:/Dataset/202101_withDinghu/temp/Dataset/202202datasetV1_256/wangmaogou/selectedpatch/test/basicID16_crop\W0_1596_2508.png
Done!
open tif file succeed
open tif file succeed
3 1
uint16 uint16
(256, 512, 3)
uint16
E:/Dataset/202101_withDinghu/temp/Dataset/202202datasetV1_256/wangmaogou/selectedpatch/test/basicID16_crop\W0_1824_1596.png
Done!
open tif file succeed
open tif file succeed
3 1
uint16 uint16
(256, 512, 3)
uint16
E:/Dataset/202101_withDinghu/temp/Dataset/202202datasetV1_256/wangmaogou/selectedpatch/test/basicID16_crop\W0_1824_1824.png
Done!
open tif file succeed
open tif file succeed
3 1
uint16 uint16
(256, 512, 3)
uint16
E:/Dataset/202101_withDinghu/temp/Dataset/202202datasetV1_256/wan

Done!
open tif file succeed
open tif file succeed
3 1
uint16 uint16
(256, 512, 3)
uint16
E:/Dataset/202101_withDinghu/temp/Dataset/202202datasetV1_256/wangmaogou/selectedpatch/test/basicID16_crop\W0_2736_2964.png
Done!
open tif file succeed
open tif file succeed
3 1
uint16 uint16
(256, 512, 3)
uint16
E:/Dataset/202101_withDinghu/temp/Dataset/202202datasetV1_256/wangmaogou/selectedpatch/test/basicID16_crop\W0_2964_2280.png
Done!
open tif file succeed
open tif file succeed
3 1
uint16 uint16
(256, 512, 3)
uint16
E:/Dataset/202101_withDinghu/temp/Dataset/202202datasetV1_256/wangmaogou/selectedpatch/test/basicID16_crop\W0_2964_2508.png
Done!
open tif file succeed
open tif file succeed
3 1
uint16 uint16
(256, 512, 3)
uint16
E:/Dataset/202101_withDinghu/temp/Dataset/202202datasetV1_256/wangmaogou/selectedpatch/test/basicID16_crop\W0_2964_2736.png
Done!


# Move files that meet user requirements to new path
## select files.

In [35]:
!python General/SelectPatch.py

open tif file succeed
34381
open tif file succeed
open tif file succeed
10874
open tif file succeed
65536
open tif file succeed
59535
open tif file succeed
open tif file succeed
open tif file succeed
65536
open tif file succeed
4514
open tif file succeed
8080
open tif file succeed
65536
open tif file succeed
open tif file succeed
11695
open tif file succeed
64889
open tif file succeed
65536
open tif file succeed
65536
open tif file succeed
59828
open tif file succeed
57411
open tif file succeed
65536
open tif file succeed
65536
open tif file succeed
38674
open tif file succeed
39602
open tif file succeed
9403
open tif file succeed
65536
open tif file succeed
54856
open tif file succeed
47089
open tif file succeed
55085
open tif file succeed
22547
open tif file succeed
open tif file succeed
65534
open tif file succeed
41672
open tif file succeed
65536
open tif file succeed
65536
open tif file succeed
35583
open tif file succeed
64299
open tif file succeed
5794
open tif file succeed
2301

# Produce shadow maps

In [1]:
import numpy as np
from osgeo import gdal 
import os
import imageio
import glob
import cv2


In [20]:
# in TiffReadClipWrite.py
'''
readtiff2array: read tif file and output an numpy array
calculateTransform: calculating 'geotrans' affine transformation 
clipgeotiff_topatch: clip the complete data to small patches. (nonrandom)
'''
def readtiff2array(oriPath):
    in_ds = gdal.Open(oriPath)
    print("open tif file succeed")

    # 读取原图中的每个波段
    row = in_ds.RasterYSize  # 行
    col = in_ds.RasterXSize  # 列
    band = in_ds.RasterCount  # 波段
    geoTrans = in_ds.GetGeoTransform()
    geoPro = in_ds.GetProjection()

    # specific datatype
#     newDataType = gdal.GDT_UInt16;
#     data = np.zeros([row, col, band], newDataType)  # 建立数组保存读取的tiff

    
    # according to the input datatype
    datatype_index = in_ds.GetRasterBand(1).DataType
    datatype = gdal.GetDataTypeName(datatype_index)
    if 'GDT_Byte' in datatype:
        newDataType = 'int8'
    elif 'GDT_UInt16' in datatype:
        newDataType = 'int16'
    else:
        newDataType = 'float32'
    data = np.zeros([row, col, band], newDataType)  # 建立数组保存读取的tiff

    for i in range(band):
        dt = in_ds.GetRasterBand(i + 1)
        # 从每个波段中裁剪需要的矩形框内的数据
        data[:, :, i] = dt.ReadAsArray(0, 0, col, row)

    del in_ds
    
    return data, [band, datatype, geoTrans, geoPro]
    # data shape = HWC
'''
binary segmentation
according to a user-defined threshold
'''

def shadowextractor(inputdata, savepath, filename, inverse=False, threshold=150, ADAPTIVE=False):

    # shadow == 1
    os.makedirs(savePath, exist_ok=True)
    savefilePath = os.path.join(savepath, filename)

    HSV_img = cv2.cvtColor(inputdata, cv2.COLOR_BGR2HSV)
    channel = HSV_img[:, :, 2]
    
    if ADAPTIVE is True:
        # 自适应阈值
        blocksize = 15
        C=1
        retShadow, Shadowimg =  cv2.adaptiveThreshold(channel, 1, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, blocksize, C)
    else:
        # 预先设定阈值
        retShadow, Shadowimg = cv2.threshold(channel, threshold, 255, cv2.THRESH_BINARY_INV)

    cv2.imwrite(savefilePath, Shadowimg)

    # inverse shadow
    # shadow == 0
    if inverse is True:
        inverse_savePath = savePath.replace('shadow_crop', 'invshadow_crop')
        os.makedirs(inverse_savePath, exist_ok=True)
        retInverse, Inverseimg = cv2.threshold(channel, threshold, 255, cv2.THRESH_BINARY)

        inverse_savefilePath = os.path.join(inverse_savePath, filename)
        cv2.imwrite(inverse_savefilePath, Inverseimg)

    print("done")


In [21]:
imagePath = "E:/Dataset/202101_withDinghu/temp/Dataset/202201datasetV2/size224/s2/test/image_crop"
savePath = "E:/Dataset/202101_withDinghu/temp/Dataset/202201datasetV2/size224/s2/test/shadow_testV"

# 遍历文件夹
from_names = glob.glob(os.path.join(imagePath, "*.tif"))
for i in range(len(from_names)):
    basename = os.path.basename(from_names[i])
    
    inputdata, para = readtiff2array(os.path.join(imagePath, basename))
    shadowextractor(inputdata, savePath, basename, inverse=True)

open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
open tif file succeed
done
o